# Bulletin de suivi hydrologique

Parties automatisées de l'édition du bulletin : suivi des réserves stockées



In [ ]:
## imports des packages utiles

import numpy as np
import pandas as pd

# graphiques
import matplotlib.pyplot as plt
import seaborn as sns

# partie interactive
import ipywidgets as widgets
from IPython.display import display


# pour jupyter notebook
%matplotlib widget

# tailles figures
largeur = 10
hauteur = 7

## Suivi des réserves en eau

Le traitement se fait en deux parties :

- récupération des données depuis la base de données de aGHyre
- mise en forme des données pour présentation

In [ ]:
# lancement du script de récupération des données sur aGHyre

inp="./donnees/recuperer_donnees_aghyre_v1.ini"
%run ./scripts/Aghyre/recuperer_donnees_aghyre_v1.py "{inp}"

# le fichier "inp" contient les informations pour l'interprétation du suivi :
# la correspondance nom de réservoir - code rubrique
fic_id_rub_m3 = "./donnees/rubriques_volume_utile_m3.csv"
fic_id_rub_Mm3 = "./donnees/rubriques_volume_utile_Mm3.csv"
# la chronique des volumes utiles des réservoirs
fic_vol_utile_m3 = "./donnees/chroniques/chronique_rubriques_volume_utile_m3.csv"
fic_vol_utile_Mm3 = "./donnees/chroniques/chronique_rubriques_volume_utile_Mm3.csv"


In [3]:
# lecture des données

# idenfifiants des rubriques
df_id_rub_m3  = pd.read_csv(fic_id_rub_m3, sep=';')
df_id_rub_Mm3 = pd.read_csv(fic_id_rub_Mm3, sep=';')
df_id_rub = pd.concat([df_id_rub_Mm3, df_id_rub_m3], axis=0)
# index par id de rubrique comme valeur numérique
df_id_rub = df_id_rub.set_index('id_rubrique')

# chroniques : conversion de tout en Mm3 et réunion des données
df_vol_utile_m3 = pd.read_csv(fic_vol_utile_m3, sep=';', index_col=0, parse_dates=True) * 1.e-6
df_vol_utile_Mm3 = pd.read_csv(fic_vol_utile_Mm3, sep=';', index_col=0, parse_dates=True)
df_vol_utile = df_vol_utile_Mm3.join(df_vol_utile_m3, how='outer')

# suppression des lignes vides
df_vol_utile = df_vol_utile.dropna(axis=0, how='all')

# construction des données au pas de temps journalier
df_vol_utile = df_vol_utile.resample('D').mean()
df_vol_utile = df_vol_utile.resample('MS').first()

# somme de tous les volumes
df_vol_total = pd.DataFrame(df_vol_utile.apply(np.nansum, axis=1), index=df_vol_utile.index, columns=['volume_Mm3'])

# pour regrouper les graphes par année
df_vol_annees = df_vol_total.pivot_table(index=df_vol_total.index.month,columns=df_vol_total.index.year, values='volume_Mm3')


In [ ]:
# vue matricielle de la complétude des données en fonction du temps

fig, ax = plt.subplots(1,1)
fig.set_figwidth(largeur)
fig.set_figheight(hauteur)

# zoom sur certaines années
df_zoom = df_vol_utile.loc["2015":"2024"]

# nom des colonnes : avec nom des réservoirs
idx_nom_reservoirs = df_id_rub.to_dict(orient='dict')['nom']
df_zoom.columns = df_zoom.columns.map(float)
df_zoom = df_zoom.rename(columns=idx_nom_reservoirs)

sns.heatmap(df_zoom.notna(),
            cmap='YlGnBu',
            cbar=False,
            yticklabels=12,
            ax=ax
            )
# axes des dates
ytick_labels = [t.strftime('%Y-%m') for t in df_zoom.index[0::12]]
ax.set_yticklabels(ytick_labels)
ax.tick_params(axis='x', bottom=False, top=True, labelbottom=False, labeltop=True)
ax.set_xticklabels(df_zoom.columns, rotation=45, ha='left')

fig.tight_layout()


In [ ]:
# affichage
# figure
fig, ax = plt.subplots(1,1)
fig.set_figwidth(largeur)
fig.set_figheight(hauteur)

# depuis 2021
df_vol_annees.columns.name = 'année'
df_vol_annees.loc[:,2021:].plot(ax=ax, legend=True)

# limites
ax.set_ylim(0,160)
ax.set_title("Evolution du volume global des réserves en eau VNF (2021-2025)")
ax.set_xlabel('')
ax.set_ylabel("Volume global VNF ($Mm^3$)")
ax.set_xticks(df_vol_annees.index, df_vol_utile.index.map(lambda t:t.strftime('%B')).unique(), rotation=45, ha='right')

ax.grid(axis='both', color='grey', linestyle='--', linewidth=0.5, alpha=0.5)

fig.tight_layout()



In [6]:
# synthèse de l'état des réserves pour le bilan mensuel

# pour estimer le taux de remplissage des barrages il faut récupérer les volumes max utiles.
fic_caracteristiques_reservoirs = r"C:\Users\alain.gauthier\OneDrive - Voies navigables de France\DGH\13_Animation_Reseau_Secheresse\Suivi des réserves en eau\Caractéristiques des réserves.xlsx"

# lecture du fichier
df_carac_reservoir = pd.read_excel(fic_caracteristiques_reservoirs,
                                   sheet_name="Réservoirs",
                                   skiprows=2,
                                   header=1,
                                   )

# on conserve les données présentes dans aghyre
df_carac_reservoir = df_carac_reservoir.dropna(subset='ID Aghyre - VMJ utile',
                                               axis=0,
                                               how='any')

# index par identifiant de rubrlique sur Aghyre

df_carac_reservoir = df_carac_reservoir.set_index('ID Aghyre - VMJ utile')
df_carac_reservoir.index = df_carac_reservoir.index.map(lambda x: f'{int(x)}')

# synthèse à une date donnée
date_synthese = dt.date(2025,5,1)
date_prec = date_synthese - pd.DateOffset(months=1)
# formatage dates
date_synthese = date_synthese.strftime('%Y-%m')

# remplissage à la date demandée
df_remplissage = df_vol_utile.loc[date_synthese].T
df_remplissage.columns = ['Volume utile en $Mm^3$']

# noms des réservoirs
df_remplissage['Barrages réservoirs'] = df_carac_reservoir['Barrages réservoirs']
# capacité max utile
df_remplissage['Capacité maximale utile en $Mm^3$'] = df_carac_reservoir['Capacité maximale utile (en Mm3)']

# indicateur statistique regroupé par mois
df_group_mois = df_vol_utile.groupby(df_vol_utile.index.month)
# valeur de référence sur 10 ans avec autorisation de manque de 1 valeur manquante (10%)
df_10_ans = df_group_mois.rolling(window=10, min_periods=9).mean()
df_10_ans.index = df_10_ans.index.get_level_values(1)
df_10_ans = df_10_ans.sort_index()

# valeur de référence sur 10 ans (à la date de synthèse)
df_remplissage["Valeur de référence sur 10 ans (volume de remplissage en $Mm^3$)"] = df_10_ans.loc[date_synthese].T

# taux de remplissage utile
df_remplissage["Taux de remplissage utile"] = df_remplissage['Volume utile en $Mm^3$'] / df_remplissage['Capacité maximale utile en $Mm^3$']

# arrangement des colonnes
df_remplissage = df_remplissage[['Barrages réservoirs',
                                'Capacité maximale utile en $Mm^3$',
                                "Valeur de référence sur 10 ans (volume de remplissage en $Mm^3$)",
                                'Volume utile en $Mm^3$',
                                "Taux de remplissage utile"]]

# Tendance d'évolution par rapport au mois précédent

# taux de remplissage mois précédent
df_rempl_prec = df_vol_utile.loc[date_prec.strftime('%Y-%m')].T.apply(lambda x: x / df_remplissage.loc[x.index, 'Capacité maximale utile en $Mm^3$'])
df_remplissage['tendance'] = df_remplissage['Taux de remplissage utile'] - df_rempl_prec[date_prec]

# pour finalisation
df_remplissage["Voies d'eau"] = df_carac_reservoir["Voies d'eau"]
df_remplissage["Voies d'eau"] = df_remplissage["Voies d'eau"].ffill()
# Attention au nom de la colonne : fragile /!\
df_remplissage['DT'] = df_carac_reservoir['Est']
df_remplissage['DT'] = df_remplissage['DT'].ffill()


In [7]:
# mise en forme

# index
df_visu = df_remplissage.reset_index()
df_visu = df_visu.set_index(['DT', "Voies d'eau", 'index'])

# emoji selon la valeur de référence
table_emoji = ['🔴', '🟡', '🟢']
df_visu['emoji'] = df_visu.apply(lambda x: 0 if x['Volume utile en $Mm^3$']< 0.8*x["Valeur de référence sur 10 ans (volume de remplissage en $Mm^3$)"]
                                 else 1 if x['Volume utile en $Mm^3$']< x["Valeur de référence sur 10 ans (volume de remplissage en $Mm^3$)"]
                                 else 2,
                                 axis=1)
df_visu['emoji'] = df_visu['emoji'].apply(lambda x: table_emoji[x])

# flèche tendance
table_fleches = ['↘','→','↗']
df_visu['tendance'] = df_visu.apply(lambda x:0 if x['tendance']<= -0.03
                                    else 1 if x['tendance']< 0.03
                                    else 2,
                                    axis=1)
df_visu['tendance'] = df_visu['tendance'].apply(lambda x: table_fleches[x])

# on abandonne les index de rubrique pour l'affichage
#df_visu.index = df_visu.index.droplevel(2)
# affichage
    #.set_properties(**{'background-color': '#ECE3FF','color': 'black'}) \
df_visu.style \
    .format(precision=2) \
    .bar(subset=['Taux de remplissage utile'], vmin=0, vmax=1, cmap="RdYlGn") \
    .map( lambda v: 'color:red;' if v == table_fleches[0]
               else 'color:orange;' if v == table_fleches[1]
               else 'color:green;', subset=['tendance']) \
    .hide(level=2, axis=0)